In [1]:
import pycisTopic
pycisTopic.__version__

'0.1.dev300+g7494158'

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [3]:
import pickle
import pandas as pd

In [4]:
import copy

In [5]:
import os
wdir = '/lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/'
os.chdir( wdir )

In [6]:
# create output directory:
f_final_dir = os.path.join(wdir, 'downstream_analysis')
if not os.path.exists(f_final_dir):
    os.makedirs(f_final_dir)

In [7]:
# display names of each sample. NOTE: the samples will be dislpayed in the order of this dict!
from collections import OrderedDict
alias_dict = OrderedDict({
    "Broad_1": "BioRad ATAC 1",
    "Broad_2": "BioRad ATAC 2",
    "Stanford_1": "10x ATAC A1",
    "Stanford_2": "10x ATAC A2",
    "VIB_1": "10x ATAC B1",
    "VIB_2": "10x ATAC B2",
    "CNAG_1": "10x ATAC C1",
    "CNAG_2": "10x ATAC C2",
    "Broad_mito_1": "10x mtATAC 1",
    "Broad_mito_2": "10x mtATAC 2",
    "Sanger_1": "10x Multiome 1",
    "Sanger_2": "10x Multiome 2",
    "VIB_Hydrop_1": "Hydrop ATAC 1",
    "VIB_Hydrop_2": "Hydrop ATAC 2",
    "s3atac": "s3 ATAC",
})

color_dict = OrderedDict({
    "Broad_mito_1": "#9467bd",
    "Broad_mito_2": "#c5b0d5",
    "CNAG_1": "#d62728",
    "CNAG_2": "#ff9896",
    "Broad_1": "#1f77b4",
    "Broad_2": "#aec7e8",
    "Sanger_1": "#8c564b",
    "Sanger_2": "#c49c94",
    "Stanford_1": "#ff7f0e",
    "Stanford_2": "#ffbb78",
    "VIB_1": "#2ca02c",
    "VIB_2": "#98df8a",
    "VIB_Hydrop_1": "#e377c2",
    "VIB_Hydrop_2": "#f7b6d2",
    "s3atac": "#7f7f7f",
})

tech_dict = OrderedDict({
    "Broad_1": "BioRad ATAC",
    "Broad_2": "BioRad ATAC",
    "Stanford_1": "10x ATAC A",
    "Stanford_2": "10x ATAC A",
    "VIB_1": "10x ATAC B",
    "VIB_2": "10x ATAC B",
    "CNAG_1": "10x ATAC C",
    "CNAG_2": "10x ATAC C",
    "Broad_mito_1": "10x mtATAC",
    "Broad_mito_2": "10x mtATAC",
    "Sanger_1": "10x Multiome",
    "Sanger_2": "10x Multiome",
    "VIB_Hydrop_1": "Hydrop ATAC",
    "VIB_Hydrop_2": "Hydrop ATAC",
    "s3atac": "s3 ATAC",
})

## Load the cisTopic objects

In [8]:
import glob
f_cto_dir = 'pycistopic_consensus_peaks/cistopic_objs__consensus'

glob.glob(f_cto_dir + '/*__cistopic_obj_metadata_annotated_dimreduc_df.pkl')

['pycistopic_consensus_peaks/cistopic_objs__consensus/s3atac__cistopic_obj_metadata_annotated_dimreduc_df.pkl',
 'pycistopic_consensus_peaks/cistopic_objs__consensus/CNAG_2__cistopic_obj_metadata_annotated_dimreduc_df.pkl',
 'pycistopic_consensus_peaks/cistopic_objs__consensus/VIB_2__cistopic_obj_metadata_annotated_dimreduc_df.pkl',
 'pycistopic_consensus_peaks/cistopic_objs__consensus/VIB_1__cistopic_obj_metadata_annotated_dimreduc_df.pkl',
 'pycistopic_consensus_peaks/cistopic_objs__consensus/Sanger_2__cistopic_obj_metadata_annotated_dimreduc_df.pkl',
 'pycistopic_consensus_peaks/cistopic_objs__consensus/Broad_2__cistopic_obj_metadata_annotated_dimreduc_df.pkl',
 'pycistopic_consensus_peaks/cistopic_objs__consensus/VIB_Hydrop_1__cistopic_obj_metadata_annotated_dimreduc_df.pkl',
 'pycistopic_consensus_peaks/cistopic_objs__consensus/Broad_mito_2__cistopic_obj_metadata_annotated_dimreduc_df.pkl',
 'pycistopic_consensus_peaks/cistopic_objs__consensus/Broad_mito_1__cistopic_obj_metadata_a

In [9]:
cistopic_obj_dict = {}
for key in alias_dict.keys():
    f_cto = os.path.join(wdir, f_cto_dir, key + '__cistopic_obj_metadata_annotated_dimreduc_df.pkl')
    if(os.path.isfile(f_cto)):
        with open(f_cto, 'rb') as f:
            cistopic_obj_dict[key] = pickle.load(f)
        print(f"Loaded filtered cistopic object {key}")
    else:
        print(f"file {f_cto} doesn't exist")

Loaded filtered cistopic object Broad_1


Loaded filtered cistopic object Broad_2
Loaded filtered cistopic object Stanford_1


Loaded filtered cistopic object Stanford_2


Loaded filtered cistopic object VIB_1


Loaded filtered cistopic object VIB_2


...

Loaded filtered cistopic object Broad_mito_2


Loaded filtered cistopic object Sanger_1


Loaded filtered cistopic object Sanger_2
Loaded filtered cistopic object VIB_Hydrop_1


Loaded filtered cistopic object VIB_Hydrop_2


Loaded filtered cistopic object s3atac


#### Save/load
Only necessary if you have not calculated accessibility yet.

In [10]:
f_out = os.path.join(f_final_dir, 'region_bin_topics.pkl')
if os.path.isfile(f_out):
    with open(f_out, 'rb') as f:
        region_bin_topics_dict = pickle.load(f)
    print(f"Loaded {f_out}")

Loaded /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_bin_topics.pkl


In [11]:
region_bin_topics_dict.keys()

dict_keys(['VIB_Hydrop_1', 'VIB_Hydrop_2', 'Broad_1', 'Broad_2', 'Broad_mito_1', 'Broad_mito_2', 'CNAG_1', 'CNAG_2', 's3atac', 'Sanger_1', 'Sanger_2', 'Stanford_1', 'Stanford_2', 'VIB_1', 'VIB_2'])

In [12]:
f_out = os.path.join(f_final_dir, 'binarized_cell_topic.pkl')
if os.path.isfile(f_out):
    with open(f_out, 'rb') as f:
        binarized_cell_topic_dict = pickle.load(f)
    print(f"Loaded {f_out}")

Loaded /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/binarized_cell_topic.pkl


In [13]:
binarized_cell_topic_dict.keys()

dict_keys(['VIB_Hydrop_1', 'VIB_Hydrop_2', 'Broad_1', 'Broad_2', 'Broad_mito_1', 'Broad_mito_2', 'CNAG_1', 'CNAG_2', 's3atac', 'Sanger_1', 'Sanger_2', 'Stanford_1', 'Stanford_2', 'VIB_1', 'VIB_2'])

In [14]:
f_out = os.path.join(f_final_dir, 'imputed_acc_obj.pkl')
if os.path.isfile(f_out):
    with open(f_out, 'rb') as f:
        imputed_acc_obj_dict = pickle.load(f)
    print(f"Loaded {f_out}")

Loaded /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/imputed_acc_obj.pkl


In [15]:
imputed_acc_obj_dict.keys()

dict_keys(['VIB_Hydrop_1', 'VIB_Hydrop_2', 'Broad_1', 'Broad_2', 'Broad_mito_1', 'Broad_mito_2', 'CNAG_1', 'CNAG_2', 's3atac', 'Sanger_1', 'Sanger_2', 'Stanford_1', 'Stanford_2', 'VIB_1', 'VIB_2'])

In [16]:
f_out = os.path.join(f_final_dir, 'markers_dict.pkl')
if os.path.isfile(f_out):
    with open(f_out, 'rb') as f:
        markers_dict_dict = pickle.load(f)
    print(f"Loaded {f_out}")

Loaded /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/markers_dict.pkl


In [17]:
markers_dict_dict.keys()

dict_keys(['VIB_Hydrop_1', 'VIB_Hydrop_2', 'Broad_1', 'Broad_2', 'Broad_mito_1', 'Broad_mito_2', 'CNAG_1', 'CNAG_2', 's3atac', 'Sanger_1', 'Sanger_2', 'Stanford_1', 'Stanford_2', 'VIB_1', 'VIB_2'])

## Infer or load gene activity

In [18]:
import pyranges as pr
import requests
import pybiomart as pbm

In [19]:
from pycisTopic.gene_activity import get_gene_activity
from pycisTopic.diff_features import find_diff_features

In [20]:
cistopic_obj_dict['VIB_Hydrop_1'].cell_data

,sample_id,SCREEN_fmx_sample,SCREEN_Dupl_rate,cisTopic_nr_frag,Unique_nr_frag_in_regions,cisTopic_log_nr_frag,Total_nr_frag,pycisTopic_leiden_10_0.6,Log_total_nr_frag,Dupl_nr_frag,...,SCREEN_Log_unique_nr_frag,barcode,Total_nr_frag_in_regions,Unique_nr_frag,pycisTopic_leiden_10_0.8,pycisTopic_leiden_10_1.0,UMAP_1,UMAP_2,tSNE_1,tSNE_2
CTCCTCACTACTTGGAATTC-12-VIB_Hydrop_1,VIB_Hydrop_1,sampleA,0.882217,8980,8828,3.953276,183600,7,5.263873,161975,...,4.334956,CTCCTCACTACTTGGAATTC-12,73932,21625,8,9,10.265941,9.402528,0.366964,-1.424251
AAGTCAAGCCCCACGTTGTG-12-VIB_Hydrop_1,VIB_Hydrop_1,sampleA,0.848606,754,751,2.877371,8250,5,3.916454,7001,...,3.096562,AAGTCAAGCCCCACGTTGTG-12,4906,1249,4,4,-3.301128,10.819255,-14.013913,-13.247282
TAACTCACAGGCGTGGATCA-12-VIB_Hydrop_1,VIB_Hydrop_1,sampleA,0.794502,1381,1349,3.140194,9713,2,3.987353,7717,...,3.300161,TAACTCACAGGCGTGGATCA-12,6737,1996,8,7,13.733324,5.989512,6.781381,-12.628861
GTAGATACGAGATTCCGGAG-12-VIB_Hydrop_1,VIB_Hydrop_1,sampleB,0.801536,5284,5161,3.722963,37901,7,4.578651,30379,...,3.876333,GTAGATACGAGATTCCGGAG-12,26642,7522,8,9,10.874220,8.800022,2.932455,-5.112085
GAACTCAACCTAGTGTGTTG-12-VIB_Hydrop_1,VIB_Hydrop_1,sampleA,0.683983,2797,2759,3.446692,13167,5,4.119487,9006,...,3.619198,GAACTCAACCTAGTGTGTTG-12,8731,4161,4,4,-3.414306,10.987544,-11.629621,-11.725644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ATAAGCAGTCCAGTGTGGAA-12-VIB_Hydrop_1,VIB_Hydrop_1,sampleA,0.729045,1226,1213,3.08849,6156,0,3.789299,4488,...,3.222196,ATAAGCAGTCCAGTGTGGAA-12,4638,1668,5,3,-2.417925,8.840957,-27.255746,7.378752
GTCAATCCACGAGTAACAGG-11-VIB_Hydrop_1,VIB_Hydrop_1,sampleB,0.807930,787,755,2.895975,5498,3,3.740205,4442,...,3.023664,GTCAATCCACGAGTAACAGG-11,4044,1056,9,8,-1.509366,13.997252,-23.925141,-27.848061
CAATTGGAGAGTGCGAACCT-11-VIB_Hydrop_1,VIB_Hydrop_1,sampleB,0.799292,679,670,2.83187,4519,3,3.655042,3612,...,2.957607,CAATTGGAGAGTGCGAACCT-11,3520,907,6,6,-3.349487,15.216449,-38.930815,-23.793503
GCACACAGTCCATAATGCCT-11-VIB_Hydrop_1,VIB_Hydrop_1,sampleA,0.675778,960,927,2.982271,3951,3,3.596707,2670,...,3.107549,GCACACAGTCCATAATGCCT-11,2950,1281,6,6,-3.365353,14.679255,-35.462180,-19.926695


In [21]:
f_out = os.path.join(f_final_dir, 'gene_act_dict.pkl')
if os.path.isfile(f_out):
    print(f"Loading {f_out}")
    with open(f_out, 'rb') as f:
        gene_act_dict = pickle.load(f)
else:
    from pycisTopic.gene_activity import get_gene_activity
    from pycisTopic.diff_features import find_diff_features
    # For human
    dataset = pbm.Dataset(name='hsapiens_gene_ensembl',  host='http://www.ensembl.org')
    annot = dataset.query(attributes=['chromosome_name', 'start_position', 'end_position', 'strand', 'external_gene_name', 'transcription_start_site', 'transcript_biotype'])
    annot['Chromosome/scaffold name'] = 'chr' + annot['Chromosome/scaffold name'].astype(str)
    annot.columns=['Chromosome', 'Start', 'End', 'Strand', 'Gene','Transcription_Start_Site', 'Transcript_type']
    annot = annot[annot.Transcript_type == 'protein_coding']
    annot.Strand[annot.Strand == 1] = '+'
    annot.Strand[annot.Strand == -1] = '-'
    pr_annotation = pr.PyRanges(annot.dropna(axis = 0))
    pr_annotation

    # get chromosome sizes (hg38)
    target_url = 'http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.chrom.sizes'
    chromsizes = pd.read_csv(target_url, sep='\t', header=None)
    chromsizes.columns = ['Chromosome', 'End']
    chromsizes['Start'] = [0]*chromsizes.shape[0]
    chromsizes = chromsizes.loc[:,['Chromosome', 'Start', 'End']]
    chromsizes = pr.PyRanges(chromsizes)
    chromsizes

    gene_act_dict = {}
    for key in cistopic_obj_dict.keys():
        print(key)
        gene_act, weights = get_gene_activity(
            imputed_acc_obj_dict[key], # Region-cell probabilities
            pr_annotation, # Gene annotation
            chromsizes, # Chromosome size
            use_gene_boundaries=True, # Whether to use the whole search space or stop when encountering another gene
            upstream=[1000, 100000], # Search space upstream. The minimum means that even if there is a gene right next to it 
                                     #these bp will be taken (1kbp here)
            downstream=[1000,100000], # Search space downstream
            distance_weight=True, # Whether to add a distance weight (an exponential function, the weight will decrease with distance)
            decay_rate=1, # Exponent for the distance exponential funciton (the higher the faster will be the decrease)
            extend_gene_body_upstream=10000, # Number of bp upstream immune to the distance weight (their value will be maximum for 
                                             #this weight)
            extend_gene_body_downstream=500, # Number of bp downstream immune to the distance weight
            gene_size_weight=False, # Whether to add a weights based on the length of the gene
            gene_size_scale_factor='median', # Dividend to calculate the gene size weigth. Default is the median value of all genes
                                             #in the genome
            remove_promoters=False, # Whether to remove promoters when computing gene activity scores
            average_scores=True, # Whether to divide by the total number of region assigned to a gene when calculating the gene 
                                 # activity score
            scale_factor=1, # Value to multiply for the final gene activity matrix
            extend_tss=[10,10], # Space to consider a promoter
            gini_weight = True, # Whether to add a gini index weigth. The more unique the region is, the higher this weight will be
            return_weights= True, # Whether to return the final weights
            project='Gene_activity') # Project name for the gene activity object
        gene_act_dict[key] = copy.copy(gene_act)

    with open(f_out, 'wb') as f:
        pickle.dump(gene_act_dict, f)
    print(f'saved {f_out}')

Loading /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_dict.pkl


In [22]:
gene_act_dict

{'Broad_1': <pycisTopic.diff_features.CistopicImputedFeatures at 0x2aec87c57e80>,
 'Broad_2': <pycisTopic.diff_features.CistopicImputedFeatures at 0x2aec83449970>,
 'Stanford_1': <pycisTopic.diff_features.CistopicImputedFeatures at 0x2aec82441d00>,
 'Stanford_2': <pycisTopic.diff_features.CistopicImputedFeatures at 0x2aec834efb20>,
 'VIB_1': <pycisTopic.diff_features.CistopicImputedFeatures at 0x2aec834ef580>,
 'VIB_2': <pycisTopic.diff_features.CistopicImputedFeatures at 0x2aec833d34f0>,
 'CNAG_1': <pycisTopic.diff_features.CistopicImputedFeatures at 0x2aec833d3a90>,
 'CNAG_2': <pycisTopic.diff_features.CistopicImputedFeatures at 0x2aec833d3220>,
 'Broad_mito_1': <pycisTopic.diff_features.CistopicImputedFeatures at 0x2aeb4f7515e0>,
 'Broad_mito_2': <pycisTopic.diff_features.CistopicImputedFeatures at 0x2aec833da340>,
 'Sanger_1': <pycisTopic.diff_features.CistopicImputedFeatures at 0x2aec833da8e0>,
 'Sanger_2': <pycisTopic.diff_features.CistopicImputedFeatures at 0x2aec833dabb0>,
 'VI

In [23]:
for key in gene_act_dict.keys():
    f_out = os.path.join(f_final_dir, key + '__libds__gene_act.pkl')
    print(f'printing sample {key} at {f_out}')
    if os.path.isfile(f_out):
        print(f"\talready existed!")
    else:
        with open(f_out, 'wb') as f:
            pickle.dump(gene_act_dict[key], f)
        print("\tdone")

printing sample Broad_1 at /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/Broad_1__libds__gene_act.pkl
	already existed!
printing sample Broad_2 at /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/Broad_2__libds__gene_act.pkl
	already existed!
printing sample Stanford_1 at /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/Stanford_1__libds__gene_act.pkl
	already existed!
printing sample Stanford_2 at /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/Stanford_2__libds__gene_act.pkl
	already existed!
printing sample VIB_1 at /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/VIB_1__libds

## Export to loom

In [24]:
f_out = os.path.join(f_final_dir, 'gene_act_dict.pkl')
if os.path.isfile(f_out):
    with open(f_out, 'rb') as f:
        gene_act_dict = pickle.load(f)
    print(f"Loaded {f_out}")

Loaded /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_dict.pkl


In [25]:
gene_act_dict.keys()

dict_keys(['Broad_1', 'Broad_2', 'Stanford_1', 'Stanford_2', 'VIB_1', 'VIB_2', 'CNAG_1', 'CNAG_2', 'Broad_mito_1', 'Broad_mito_2', 'Sanger_1', 'Sanger_2', 'VIB_Hydrop_1', 'VIB_Hydrop_2', 's3atac'])

In [26]:
f_gene_loom_dir = os.path.join(f_final_dir, 'gene_act_loom')
if not os.path.exists(f_gene_loom_dir):
    os.makedirs(f_gene_loom_dir)

In [27]:
from pycisTopic.loom import (
    export_region_accessibility_to_loom,
    export_gene_activity_to_loom
)

### Gene activity

In [28]:
from ctxcore.genesig import Regulon

# generate a dummy regulon (required for export_gene_activity_to_loom):
phreg = Regulon(
        name='placeholder regulon',
        gene2weight={'phreg': 1.0},
        transcription_factor="phreg",
        gene2occurrence={"phreg": 1},
    )

In [29]:
alias_dict_nospace = {}
for key in alias_dict.keys():
    alias_dict_nospace[key] = alias_dict[key].replace(' ','_')

In [30]:
for key in cistopic_obj_dict.keys():
    print(key)
    f_out = os.path.join(f_gene_loom_dir, alias_dict_nospace[key] + '__libds_gene_activity.loom')
    if os.path.exists(f_out):
        print(f"Skipping {f_out}: already exists.")
        continue
        
    export_gene_activity_to_loom(
        gene_activity_matrix = gene_act_dict[key],
        cistopic_obj = cistopic_obj_dict[key], 
        regulons = [phreg],
        # selected_cells = [ x.split('-')[0] + '-' + x.split('-')[1]  for x in cistopic_obj_dict[key].cell_names ], # this leaves a cell barcode of the format type 'TGCATGTCGCCGTTCCAAGA-21'
        # selected_cells = cistopic_obj_dict[key].projections['cell']['UMAP'].index.tolist(), # cflerin original 
        out_fname = f_out,
        cluster_annotation = ['consensus_cell_type', 'fmx_sample'],
        cluster_markers = {'consensus_cell_type': markers_dict_dict[key], 'fmx_sample':{}},
        tree_structure = ('scATAC', 'Gene_activity'),
        title = f'Gene activity from library-downsampled dataset {alias_dict[key]}',
        nomenclature = "hg38"
    )

Broad_1
Skipping /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_loom/BioRad_ATAC_1__libds_gene_activity.loom: already exists.
Broad_2
Skipping /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_loom/BioRad_ATAC_2__libds_gene_activity.loom: already exists.
Stanford_1
Skipping /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_loom/10x_ATAC_A1__libds_gene_activity.loom: already exists.
Stanford_2
Skipping /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_loom/10x_ATAC_A2__libds_gene_activity.loom: already exists.
VIB_1
Skipping /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samp

### Region accessibility

In [31]:
f_region_loom_dir = os.path.join(f_final_dir, 'region_acc_loom')
if not os.path.exists(f_region_loom_dir):
    os.makedirs(f_region_loom_dir)

In [32]:
for key in cistopic_obj_dict.keys():
    print(key)
    s = 'merged' if key=='libds_merged' else key
    f_out = os.path.join(f_region_loom_dir, alias_dict_nospace[key] + '__libds_region_accessibility.loom')

    if os.path.exists(f_out):
        print(f"Skipping {f_out}: already exists.")
        continue
    # Subset regions, we will use only regions in topics and DARs here to make it faster
    regions_in_topics = list(set(sum([region_bin_topics_dict[key][i].index.tolist() for i in region_bin_topics_dict[key].keys()],[])))
    regions_in_DARs = list(set(sum([markers_dict_dict[key][i].index.tolist() for i in markers_dict_dict[key].keys()],[])))
    # make sure we only take regions that actually exist in the accessibility matrix:
    selected_regions = list(set(regions_in_topics + regions_in_DARs).intersection(set(imputed_acc_obj_dict[key].feature_names)))

    # Export to loom
    export_region_accessibility_to_loom(
        accessibility_matrix = imputed_acc_obj_dict[key],
        cistopic_obj = cistopic_obj_dict[key], 
        binarized_topic_region = region_bin_topics_dict[key],
        binarized_cell_topic = binarized_cell_topic_dict[key],
        out_fname = f_out,
        selected_regions = selected_regions ,
        # selected_cells = [ x.split('-')[0] + '-' + x.split('-')[1]  for x in cistopic_obj_dict[key].cell_names ], # this leaves a cell barcode of the format type 'TGCATGTCGCCGTTCCAAGA-21'
        # selected_cells = cistopic_obj_dict[key].projections['cell']['UMAP'].index.tolist(), # cflerin original
        cluster_annotation = ['consensus_cell_type', 'SCREEN_fmx_sample'],
        cluster_markers = {'consensus_cell_type': markers_dict_dict[key], 'SCREEN_fmx_sample':{}},
        tree_structure = ('scATAC', 'Region_accessibility'),
        title = f'Region accessibility from library-downsampled dataset {alias_dict[key]}',
        nomenclature = "hg38"
    )

Broad_1
Skipping /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/BioRad_ATAC_1__libds_region_accessibility.loom: already exists.
Broad_2
Skipping /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/BioRad_ATAC_2__libds_region_accessibility.loom: already exists.
Stanford_1
Skipping /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/10x_ATAC_A1__libds_region_accessibility.loom: already exists.
Stanford_2
Skipping /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/10x_ATAC_A2__libds_region_accessibility.loom: already exists.
VIB_1
Skipping /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_githu

# open gene activity loom and fix annotations

In [33]:
import loompy as lp
import json
import copy
import glob
from collections import OrderedDict

exclude 'barcode' annotation

In [34]:
filenames = sorted(glob.glob(f_gene_loom_dir+'/*loom'))
samples = [item.replace("__libds_gene_activity.loom", "") for item in filenames]
samples = [item.replace("/lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_loom/", "") for item in samples]
loom_dict = {samples[i]: filenames[i] for i in range(len(samples))}
loom_dict = OrderedDict(loom_dict)

In [35]:
loom_dict

OrderedDict([('10x_ATAC_A1',
              '/lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_loom/10x_ATAC_A1__libds_gene_activity.loom'),
             ('10x_ATAC_A2',
              '/lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_loom/10x_ATAC_A2__libds_gene_activity.loom'),
             ('10x_ATAC_B1',
              '/lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_loom/10x_ATAC_B1__libds_gene_activity.loom'),
             ('10x_ATAC_B2',
              '/lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_loom/10x_ATAC_B2__libds_gene_activity.loom'),
             ('10x_ATAC_C1',
              '/lustre1/project/stg_00002/lcb/fdero

In [36]:
for sample in loom_dict.keys():
    file = loom_dict[sample]
    print(f'opening {file}')
    loom = lp.connect(file, mode='r+', validate=False)

    metadata = json.loads(loom.attrs['MetaData'])
    new_metadata = copy.deepcopy(metadata)
    for item in new_metadata['annotations']:
        if item['name'] == 'barcode':
            barcode_index = new_metadata['annotations'].index(item)

    if 'barcode_index' in locals():
        print(f'\tbarcode index found: {barcode_index}')
        new_metadata['annotations'].pop(barcode_index)
        del barcode_index

        new_metadata_json = json.dumps(new_metadata)
        loom.attrs['MetaData'] = new_metadata_json
        loom.close()
        print(f'\tfixed {sample}')

    else:
        print('\tno barcode annotation detected')
        loom.close()

opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_loom/10x_ATAC_A1__libds_gene_activity.loom
	no barcode annotation detected
opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_loom/10x_ATAC_A2__libds_gene_activity.loom
	no barcode annotation detected
opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_loom/10x_ATAC_B1__libds_gene_activity.loom


	no barcode annotation detected
opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_loom/10x_ATAC_B2__libds_gene_activity.loom
	no barcode annotation detected
opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_loom/10x_ATAC_C1__libds_gene_activity.loom
	no barcode annotation detected
opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_loom/10x_ATAC_C2__libds_gene_activity.loom


	no barcode annotation detected
opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_loom/10x_Multiome_1__libds_gene_activity.loom
	no barcode annotation detected
opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_loom/10x_Multiome_2__libds_gene_activity.loom
	no barcode annotation detected
opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_loom/10x_mtATAC_1__libds_gene_activity.loom


	no barcode annotation detected
opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_loom/10x_mtATAC_2__libds_gene_activity.loom
	no barcode annotation detected
opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_loom/BioRad_ATAC_1__libds_gene_activity.loom
	no barcode annotation detected
opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_loom/BioRad_ATAC_2__libds_gene_activity.loom


	no barcode annotation detected
opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_loom/Hydrop_ATAC_1__libds_gene_activity.loom
	no barcode annotation detected
opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_loom/Hydrop_ATAC_2__libds_gene_activity.loom


	no barcode annotation detected
opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_loom/s3_ATAC__libds_gene_activity.loom
	no barcode annotation detected


# remove the pesky sample_id from the non-merged looms

In [37]:
for sample in loom_dict.keys():
    file = loom_dict[sample]
    print(f'opening {file}')
    loom = lp.connect(file, mode='r+', validate=False)

    metadata = json.loads(loom.attrs['MetaData'])
    new_metadata = copy.deepcopy(metadata)
    for item in new_metadata['annotations']:
        if item['name'] == 'sample_id':
            barcode_index = new_metadata['annotations'].index(item)

    if 'barcode_index' in locals():
        print(f'\sample_id index found: {barcode_index}')
        new_metadata['annotations'].pop(barcode_index)
        del barcode_index

        new_metadata_json = json.dumps(new_metadata)
        loom.attrs['MetaData'] = new_metadata_json
        loom.close()
        print(f'\tfixed {sample}')

    else:
        print('\tno sample_id detected')
        loom.close()
    print('\n')

opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_loom/10x_ATAC_A1__libds_gene_activity.loom
	no sample_id detected


opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_loom/10x_ATAC_A2__libds_gene_activity.loom
	no sample_id detected


opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_loom/10x_ATAC_B1__libds_gene_activity.loom
	no sample_id detected


opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_loom/10x_ATAC_B2__libds_gene_activity.loom
	no sample_id detected


opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analys

# open region accessibility loom and fix annotations

In [38]:
import loompy as lp
import json
import copy
import glob
from collections import OrderedDict

exclude 'barcode' annotation

In [39]:
filenames = sorted(glob.glob(f_region_loom_dir+'/*loom'))
samples = [item.replace("__libds_region_accessibility.loom", "") for item in filenames]
samples = [item.replace("/lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/", "") for item in samples]
loom_dict = {samples[i]: filenames[i] for i in range(len(samples))}
loom_dict = OrderedDict(loom_dict)

In [40]:
loom_dict

OrderedDict([('10x_ATAC_A1',
              '/lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/10x_ATAC_A1__libds_region_accessibility.loom'),
             ('10x_ATAC_A2',
              '/lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/10x_ATAC_A2__libds_region_accessibility.loom'),
             ('10x_ATAC_B1',
              '/lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/10x_ATAC_B1__libds_region_accessibility.loom'),
             ('10x_ATAC_B2',
              '/lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/10x_ATAC_B2__libds_region_accessibility.loom'),
             ('10x_ATAC_C1',
              '

In [41]:
for sample in loom_dict.keys():
    file = loom_dict[sample]
    print(f'opening {file}')
    loom = lp.connect(file, mode='r+', validate=False)

    metadata = json.loads(loom.attrs['MetaData'])
    new_metadata = copy.deepcopy(metadata)
    for item in new_metadata['annotations']:
        if item['name'] == 'barcode':
            barcode_index = new_metadata['annotations'].index(item)

    if 'barcode_index' in locals():
        print(f'\tbarcode index found: {barcode_index}')
        new_metadata['annotations'].pop(barcode_index)
        del barcode_index

        new_metadata_json = json.dumps(new_metadata)
        loom.attrs['MetaData'] = new_metadata_json
        loom.close()
        print(f'\tfixed {sample}')

    else:
        print('\tno barcode annotation detected')
        loom.close()

opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/10x_ATAC_A1__libds_region_accessibility.loom
	no barcode annotation detected
opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/10x_ATAC_A2__libds_region_accessibility.loom
	no barcode annotation detected
opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/10x_ATAC_B1__libds_region_accessibility.loom


	no barcode annotation detected
opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/10x_ATAC_B2__libds_region_accessibility.loom
	no barcode annotation detected
opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/10x_ATAC_C1__libds_region_accessibility.loom


	no barcode annotation detected
opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/10x_ATAC_C2__libds_region_accessibility.loom
	no barcode annotation detected
opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/10x_Multiome_1__libds_region_accessibility.loom


	no barcode annotation detected
opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/10x_Multiome_2__libds_region_accessibility.loom
	no barcode annotation detected
opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/10x_mtATAC_1__libds_region_accessibility.loom


	no barcode annotation detected
opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/10x_mtATAC_2__libds_region_accessibility.loom
	no barcode annotation detected
opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/BioRad_ATAC_1__libds_region_accessibility.loom


	no barcode annotation detected
opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/BioRad_ATAC_2__libds_region_accessibility.loom
	no barcode annotation detected
opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/Hydrop_ATAC_1__libds_region_accessibility.loom


	no barcode annotation detected
opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/Hydrop_ATAC_2__libds_region_accessibility.loom
	no barcode annotation detected
opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/s3_ATAC__libds_region_accessibility.loom


	no barcode annotation detected


# remove the pesky sample_id from the non-merged looms

In [42]:
for sample in loom_dict.keys():
    file = loom_dict[sample]
    print(f'opening {file}')
    loom = lp.connect(file, mode='r+', validate=False)

    metadata = json.loads(loom.attrs['MetaData'])
    new_metadata = copy.deepcopy(metadata)
    for item in new_metadata['annotations']:
        if item['name'] == 'sample_id':
            barcode_index = new_metadata['annotations'].index(item)

    if 'barcode_index' in locals():
        print(f'\sample_id index found: {barcode_index}')
        new_metadata['annotations'].pop(barcode_index)
        del barcode_index

        new_metadata_json = json.dumps(new_metadata)
        loom.attrs['MetaData'] = new_metadata_json
        loom.close()
        print(f'\tfixed {sample}')

    else:
        print('\tno sample_id detected')
        loom.close()
    print('\n')

opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/10x_ATAC_A1__libds_region_accessibility.loom
\sample_id index found: 0
	fixed 10x_ATAC_A1


opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/10x_ATAC_A2__libds_region_accessibility.loom
\sample_id index found: 0
	fixed 10x_ATAC_A2


opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/10x_ATAC_B1__libds_region_accessibility.loom
\sample_id index found: 0
	fixed 10x_ATAC_B1


opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/10x_ATAC_B2__libds_region_accessibility.loom
\sample_id index found: 0
	fixed 10x_ATAC_B2


opening /lus

	fixed 10x_mtATAC_2


opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/BioRad_ATAC_1__libds_region_accessibility.loom
\sample_id index found: 0
	fixed BioRad_ATAC_1


opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/BioRad_ATAC_2__libds_region_accessibility.loom
\sample_id index found: 0
	fixed BioRad_ATAC_2


opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/Hydrop_ATAC_1__libds_region_accessibility.loom
	no sample_id detected


opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_acc_loom/Hydrop_ATAC_2__libds_region_accessibility.loom
	no sample_id detected


opening /lustre1/proje